# Config creator
Creates config files for all the simulations wanting to be run. These files act as the backbone of the simulations, and act as definition files to run a simulation.

The config files also assign IDs for:
- Coastlines
- Hydro runs
- Particle runs
in order to uniquely identify model runs, and for effective caching.

In [ ]:
import toml
from copy import deepcopy
from __init__ import logger
from pathlib import Path
from matplotlib.image import imread

from templating.utils import Hasher, dict_has_nones, remove_ids
import config_create as cc
from tooling import set_attr
from env import CONFIG_FOLDER, beaching_strats, coast_shapes, resolutions

# Mapping of simulations (ie. toml file names) to simulation options which specify the independent variables of the simulation.
individual_simulations = {}
for resolution in [1_000, 2_000]:
    res_str = f"{int(resolution / 1000)}km"
    coast_shape = "flat"
    beaching_scenarios = {
        f"{coast_shape}_{res_str}_naive.toml": {
            "coastline.coast_file": f"templating/coastlines/base_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "naive",
        },
        f"{coast_shape}_{res_str}_lebreton.toml": {
            "coastline.coast_file": f"templating/coastlines/base_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "Lebreton2018",
            "particle_model.beaching.fieldset_options": {
                "beaching_time_cutoff_days": 2
            },
        },
        f"{coast_shape}_{res_str}_mheen.toml": {
            "coastline.coast_file": f"templating/coastlines/base_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "Mheen2020",
            "particle_model.beaching.fieldset_options": {
                "beaching_timescale_days": 5,
            },
        },
        f"{coast_shape}_{res_str}_oninkb.toml": {
            "coastline.coast_file": f"templating/coastlines/base_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "Onink2021-beach",
            "particle_model.beaching.fieldset_options": {
                "beaching_timescale_days": 5,
            },
        },
        f"{coast_shape}_{res_str}_oninkbr.toml": {
            "coastline.coast_file": f"templating/coastlines/base_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "Onink2021-beach-resus",
            "particle_model.beaching.fieldset_options": {
                "beaching_timescale_days": 5,
                "resuspension_timescale_days": 10,
            },
        },
    }
    individual_simulations = {**individual_simulations, **beaching_scenarios}
for coast_shape in ["concave", "convex"]: # ? Not running with headland as I am not getting flow separation
    resolution = 1_000
    res_str = f"{int(resolution / 1000)}km"
    beaching_scenarios = {
        f"{coast_shape}_{res_str}_naive.toml": {
            "coastline.coast_file": f"templating/coastlines/{coast_shape}_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "naive",
        },        
        f"{coast_shape}_{res_str}_lebreton.toml": {
            "coastline.coast_file": f"templating/coastlines/{coast_shape}_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "Lebreton2018",
            "particle_model.beaching.fieldset_options": {
                "beaching_time_cutoff_days": 2
            },
        },
        f"{coast_shape}_{res_str}_mheen.toml": {
            "coastline.coast_file": f"templating/coastlines/{coast_shape}_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "Mheen2020",
            "particle_model.beaching.fieldset_options": {
                "beaching_timescale_days": 5,
            },
        },
        f"{coast_shape}_{res_str}_oninkb.toml": {
            "coastline.coast_file": f"templating/coastlines/{coast_shape}_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "Onink2021-beach",
            "particle_model.beaching.fieldset_options": {
                "beaching_timescale_days": 5,
            },
        },
        f"{coast_shape}_{res_str}_oninkbr.toml": {
            "coastline.coast_file": f"templating/coastlines/{coast_shape}_{res_str}.png",
            "coastline.dx": resolution,
            "coastline.dy": resolution,
            "particle_model.beaching.beaching_key": "Onink2021-beach-resus",
            "particle_model.beaching.fieldset_options": {
                "beaching_timescale_days": 5,
                "resuspension_timescale_days": 10,
            },
        },
    }
    individual_simulations = {**individual_simulations, **beaching_scenarios}


# Creating configs for each simulation
for toml_fname, sim_options in individual_simulations.items():
    cfg = deepcopy(cc.cfg_baseline)
    
    # Set all the options in the dictionary
    for key, value in sim_options.items():
        cfg = set_attr(cfg, key, value)

    # Validate beaching key
    cfg = cc.validate_beaching_key(cfg)
    

    # Done with configuring the baseline config. Now generate unique IDs...
    cfg = cc.generate_ids(cfg)
    cfg["toml_fname"] = toml_fname

    # Generating codes used to refer to the simulation in the thesis
    coast_code, resolution_code, beach_code = cc.get_codes(toml_fname)
    cfg["codes"] = dict(
        coast_code=coast_code,
        resolution_code=resolution_code,
        beach_code=beach_code,
        total=coast_code + resolution_code + beach_code
        )
    # Export config to toml file
    with open(CONFIG_FOLDER / toml_fname, "w") as f:
        toml.dump(cfg, f)


